<a href="https://colab.research.google.com/github/jhk0530/spider/blob/master/74_openCV%EB%A1%9C_%EC%96%BC%EA%B5%B4_%EC%9D%B8%EC%8B%9D%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## openCV로 얼굴 인식

In [0]:
!pip3 install opencv-python



교재에 있는 사진을 못 찾겠어서, girl face 의 첫 사진을 무단으로 사용했다.

<img src='https://i.pinimg.com/736x/d6/a9/57/d6a957f1d8045c9c973c12bf5968326f.jpg' width = '350'>


haar cascade file은 또 뭐지 어디서 다운로드 받는지가 궁금했는데 그지같은 블로거들은 그런거 안쓰고 뿌쓩빠쓩하고 있었다.

https://github.com/opencv/opencv/tree/master/data/haarcascades

여기서 다운로드 받음



In [18]:
import cv2
import sys

image_file = 'lucky4.jpg'

cascade_file = 'haarcascade_frontalface_alt.xml'

image = cv2.imread(image_file)
image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # 흑백으로 변환

cascade = cv2.CascadeClassifier(cascade_file)

face_list = cascade.detectMultiScale(image_gs, scaleFactor = 1.1, minNeighbors = 1, minSize = (150,150))

if len(face_list) >0 :
  print(face_list)
  color = (0,0,255)
  for face in face_list:
    x,y,w,h = face
    cv2.rectangle(image ,(x,y), (x+w,y+h), color, thickness = 8)
  cv2.imwrite('facedetect-output.PNG', image)

else :
  print('no face')

[[28 41 87 87]]



<img src='https://i.pinimg.com/736x/d6/a9/57/d6a957f1d8045c9c973c12bf5968326f.jpg' width = '300' style = 'float:left'> <img src='https://user-images.githubusercontent.com/6457691/79118077-c37e0000-7dc7-11ea-9b60-6d0fa9c9ae51.PNG' width = '300' style = 'float:left'>

근데 이게 웃기는게 얼굴 인식이 되는 이미지가 있고 안되는 이미지가 있다.


<img src='https://user-images.githubusercontent.com/6457691/79118458-f83e8700-7dc8-11ea-9835-de93a769fb0a.PNG' width = '300'> 

빡빡이도 되고 

<img src='https://user-images.githubusercontent.com/6457691/79118381-c1687100-7dc8-11ea-9227-ce9a2d351b2a.PNG' width = '300'>

지상렬 선생님도 되는데

<img src ='https://lh3.googleusercontent.com/proxy/GYzQ0xPp3gkyNdhWLUdBtLV_WfVbJjno8txxO6Wedrip8GJ9DAZgXU2hVYLz5BOcXiQ9b2YjhK3SbILAEd1DhTdAHE9N3yg9M8oV0cPOiSnUfN7OLfRMkjo' width = '300'>

왜 우리 럭키돈부리 선생님의 용안은 읽지 못하는 것일까...

너무 얼굴 크기가 작아서 그런가 잘라서 다시 해봄


<img src='https://user-images.githubusercontent.com/6457691/79118679-7bf87380-7dc9-11ea-8d96-d8ffebccc5a1.jpg' width = '300'>

마찬가지로 안됨.

뭘까 


열심히 고민을 하다가 찾았따.

```python
face_list = cascade.detectMultiScale(image_gs, scaleFactor = 1.1, minNeighbors = 1, minSize = (1,1))
```

역시 내 감이 맞았다.
opencv가 얼굴 인식 하는 부분에서 얼굴 크기가 너무 작으면 인식을 무시 하겠다는 설정이 저기 `minSize` 부분이었는데 이미지 자체에서 얼굴 크기가 너무 작아서 인식을 못한다는 내용이었던것 같다.

150, 150이었던 값을 1,1로 수정했더니 됨. 대래랜

럭키돈부리선생님... opencv의 o도 모르는 제가 해냈읍니다

<img src='https://user-images.githubusercontent.com/6457691/79118923-1789e400-7dca-11ea-8bfd-f48ac3d5e245.PNG'>

실제로 인식 되어있는 부분을 보면 다른 얼굴에 쓰여진 테두리에 비해 복선생님용안의 테두리만 두껍게, 즉 사이즈가 작게 인식되어있음을 알 수 있다.

굳


## 얼굴에 모자이크 걸기



In [25]:
import cv2, sys ,re

#
#if len(sys.argv) <= 1:
  #print('no input file')
  #quit()

#image_file = sys.argv[1]

# 원래 커맨드 상에서 입력 해야하는데, 나는 colab상에서 돌리고 있기 때문에 수정

image_file = 'man.jpg'

output_file = image_file + '-mosaic.jpg'
mosaic_rate = 5

cascade_file = 'haarcascade_frontalface_alt.xml'

image = cv2.imread(image_file)
image_gs = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cascade = cv2.CascadeClassifier(cascade_file)
face_list = cascade.detectMultiScale(image_gs, scaleFactor = 1.1, minNeighbors = 1, minSize = (100,100))

#if len(face_list) ==0:
  #print('no face')
  #quit()

print(face_list)
color = (0,0,255)
for (x,y,w,h) in face_list:
  face_img = image[y:y+h, x:x+w]
  face_img = cv2.resize(face_img, (w//mosaic_rate, h//mosaic_rate))
  face_img = cv2.resize(face_img, (w,h), interpolation = cv2.INTER_AREA)

  # 축소 후 원크기로 복귀 : 픽셀 뭉개져서 모자이크

  image[y:y+h, x:x+w] = face_img

cv2.imwrite(output_file, image)




[[362  94 262 262]]


True

<img src='https://user-images.githubusercontent.com/6457691/79119604-b7943d00-7dcb-11ea-878f-5b061a8063b1.jpg' width = '300'>
<img src='https://user-images.githubusercontent.com/6457691/79119683-eb6f6280-7dcb-11ea-8f61-c8e6d2e5de25.jpg' width = '300'>


모자이크도 잘 됨. 원래 교재에는 30 으로 모자이크를 씌웠는데, 모자이크를 씌우면 범죄자 같아서 좀 약하게 5로 씌웠다.

박서준은 좋겠다 모자이크 해도 잘생김을 감출 수가 없어서